# 📓 03_stableford_simulation.ipynb
**Purpose:** Run Monte Carlo tournament simulations using Stableford scoring based on hole-by-hole net scores.

In [ ]:
import pickle
import random
import numpy as np
from collections import defaultdict
from golf_classes import Player, PlayerRoundInfo, Tournament, Round, Team

In [ ]:

# --- Load Data ---

def load_data(filename='golf_data.pkl'):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data['players'], data['tournaments']

PICKLE_FILE = 'golf_data.pkl'  
players, tournaments = load_data(PICKLE_FILE)
print(f"✅ Loaded {len(players)} players and {len(tournaments)} tournaments.")




In [ ]:
# --- Course Hole Handicaps (Men's) ---
hole_handicap_ratings = [5, 13, 17, 3, 11, 9, 1, 15, 7, 10, 6, 18, 14, 2, 16, 4, 12, 8]

In [ ]:
# --- Compute strokes received per hole ---
def strokes_received_per_hole(player_handicap):
    strokes = [0] * 18
    for i in range(18):
        hcap = hole_handicap_ratings[i]
        if player_handicap >= hcap:
            strokes[i] += 1
        if player_handicap > 18 and player_handicap >= hcap + 18:
            strokes[i] += 1
    return strokes

In [ ]:
# --- Convert net score to Stableford points ---
def stableford_points(net_score):
    if net_score <= -2:
        return -2
    elif net_score == -1:
        return 0
    elif net_score == 0:
        return 1
    elif net_score == 1:
        return 3
    elif net_score == 2:
        return 5
    else:
        return 7

In [ ]:
# --- Calculate Stableford points for a round ---
def calculate_stableford_round(player_round):
    if not player_round.hole_scores or len(player_round.hole_scores) != 18:
        return None

    gross_scores = player_round.hole_scores
    handicap = player_round.handicap
    strokes = strokes_received_per_hole(handicap)

    total_points = 0
    for i in range(18):
        net = int(gross_scores[i]) - strokes[i]
        net_relative_to_par = net - 4  # assume par 4 for simplicity
        net_relative_to_par = round(net_relative_to_par)
        total_points += stableford_points(net_relative_to_par)

    return total_points

In [ ]:
# --- Sample Stableford score for a player ---
def sample_stableford_score(player):
    if not player.rounds:
        return None
    round_info = random.choice(player.rounds)
    return calculate_stableford_round(round_info)

In [ ]:
# --- Simulate team Stableford score ---
def simulate_team_stableford_score(team):
    player_scores = []
    for player in team.members:
        score = sample_stableford_score(player)
        if score is not None:
            player_scores.append(score)

    if len(player_scores) < 4:
        return float('-inf')  # Invalid team

    return sum(sorted(player_scores, reverse=True)[:4])

In [ ]:
# --- Simulate Stableford tournament ---
def simulate_stableford_tournament(teams, num_simulations=10000):
    team_scores = defaultdict(list)

    for _ in range(num_simulations):
        for team in teams:
            score = simulate_team_stableford_score(team)
            team_scores[team.name].append(score)

    team_averages = {name: np.mean(scores) for name, scores in team_scores.items()}
    return team_averages

In [ ]:
# --- Example Team Class and Simulation ---
class Team:
    def __init__(self, name, members):
        self.name = name
        self.members = members

# Randomly create example teams
example_teams = [
    Team("Team A", random.sample(list(players.values()), 6)),
    Team("Team B", random.sample(list(players.values()), 6))
]

# Run simulation
results = simulate_stableford_tournament(example_teams, num_simulations=10000)

# Print results
print("\n🏆 Team Stableford Averages:")
for name, avg in sorted(results.items(), key=lambda x: -x[1]):
    print(f"{name}: {avg:.2f} points")

In [ ]:
# --- Helper to find corrupted rounds ---

def find_corrupt_rounds(players):
    bad_rounds = []
    player_list = players.values() if isinstance(players, dict) else players

    for player in player_list:
        for rnd in player.rounds:
            for score in rnd.hole_scores:
                if not isinstance(score, (int, float)):
                    try:
                        float(score)
                    except (ValueError, TypeError):
                        bad_rounds.append((player.name, score))
    return bad_rounds

# Example usage
corrupt = find_corrupt_rounds(players)
for player_name, bad_score in corrupt:
    print(f"Corrupt score '{bad_score}' found for player {player_name}")

In [ ]:
# --- Helper to Print Sections of Player Data ---

def inspect_player_rounds(players, start_idx=0, num_players=5):
    """
    Print sections of player data to manually inspect hole_scores.
    Args:
        players: dict or list of Player objects
        start_idx: starting index in the player list
        num_players: how many players to show
    """
    player_list = list(players.values()) if isinstance(players, dict) else list(players)
    
    for i, player in enumerate(player_list[start_idx:start_idx+num_players], start=start_idx):
        print(f"\n=== Player {i}: {player.name} ===")
        for j, round_info in enumerate(player.rounds):
            print(f"  Round {j}: Tournament={round_info.tournament_name}, RoundNum={round_info.round_number}, Handicap={round_info.handicap}")
            print(f"    Hole Scores: {round_info.hole_scores}")
            
            # Check if any gross score is suspicious
            for k, score in enumerate(round_info.hole_scores):
                try:
                    _ = float(score)
                except (ValueError, TypeError):
                    print(f"    ⚠️ Suspicious score at hole {k+1}: '{score}'")

In [ ]:
# See first 5 players
inspect_player_rounds(players, start_idx=0, num_players=5)

# See players 10–15
#inspect_player_rounds(players, start_idx=10, num_players=5)

# See players 50–60
#inspect_player_rounds(players, start_idx=50, num_players=10)

In [ ]:
# --- Calculate and Print Player Stableford Averages ---

def calculate_and_print_player_stableford_stats(players, top_n=20, min_rounds=5):
    """
    Calculate and print top players based on average Stableford score across their rounds.
    
    Args:
        players: dict or list of Player objects
        top_n: number of top players to display
        min_rounds: minimum number of rounds a player must have to be considered
    """
    player_stats = []

    player_list = players.values() if isinstance(players, dict) else players

    for player in player_list:
        stableford_scores = []
        for rnd in player.rounds:
            score = calculate_stableford_round(rnd)
            if score is not None:
                stableford_scores.append(score)

        if len(stableford_scores) >= min_rounds:
            avg_stableford = np.mean(stableford_scores)
            std_stableford = np.std(stableford_scores)
            num_rounds = len(stableford_scores)
            player_stats.append((player.name, avg_stableford, std_stableford, num_rounds))
    
    # Sort by highest average Stableford (higher is better in Stableford!)
    player_stats.sort(key=lambda x: -x[1])

    # --- Print ---
    header = "{:<5} {:<25} {:>15} {:>12} {:>10}".format("Rank", "Player Name", "Avg Stableford", "Std Dev", "Rounds")
    print(f"\n🏌️ Top {top_n} Players by Average Stableford Score (Minimum {min_rounds} Rounds):\n")
    print(header)
    print("-" * len(header))

    for idx, (name, avg_score, std_dev, num_rounds) in enumerate(player_stats[:top_n], start=1):
        row = "{:<5} {:<25} {:>15.2f} {:>12.2f} {:>10}".format(idx, name, avg_score, std_dev, num_rounds)
        print(row)


In [ ]:
calculate_and_print_player_stableford_stats(players, top_n=20, min_rounds=5)